In [ ]:
pip install loss_landscapes

In [ ]:

import torch
import torchvision
import numpy as np
import pandas as pd 
from torchvision.transforms import transforms
from sklearn.model_selection import train_test_split
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import loss_landscapes
import copy #copying model
import tqdm #progress bar

In [ ]:
train_transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Resize((150,150)),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.RandomVerticalFlip()])
test_transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Resize((150,150))])

In [ ]:
main_image = torchvision.datasets.ImageFolder("../input/automobilepartsindentification/Automobile-parts",transform = test_transform,)

In [ ]:
nan

In [ ]:
class_name = main_image.classes
class_name

In [ ]:
total_len = len(main_image)
test_size = 0.1
spliter = int(total_len * test_size)
print(spliter)
print(total_len)

train_image,test_image = train_test_split(main_image,test_size = 0.1)
print(len(train_image))
print(len(test_image))

#Normally, the data in the test section should not be mixed, but I did not want to mess with it.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_image,shuffle= True,batch_size = 16)
test_loader = torch.utils.data.DataLoader(test_image,shuffle = False,batch_size = 8)


In [ ]:
for a,b in train_loader:
    
    
    c= torch.permute(a[0],(1,2,0))
    plt.imshow(c)
    plt.title(f"{class_name[b[0]]}")
    break
    
    

****Train****

we try weight initializating in resnet and our network

In [ ]:
model = torchvision.models.resnet50(pretrained = True) #we dont use gpu because loss landscape dont work gpu 

for param in model.parameters():
    param.requires_grad = False
model_last = model.fc.in_features

model.fc = nn.Sequential(nn.Linear(model_last,128),
                        nn.ReLU(inplace = True),
                        nn.Linear(128,len(class_name)))



scheduler:     Reduce learning rate when a metric has stopped improving.
    Models often benefit from reducing the learning rate by a factor
    of 2-10 once learning stagnates. This scheduler reads a metrics
    quantity and if no improvement is seen for a 'patience' number
    of epochs, the learning rate is reduced.

patience (int) – Number of epochs with no improvement after which learning rate will be reduced. For example, if patience = 2, then we will ignore the first 2 epochs with no improvement, and will only decrease the LR after the 3rd epoch if the loss still hasn’t improved then. Default: 10.

In [ ]:
epoch = 10
optimizer = torch.optim.Adam(model.fc.parameters(),lr = 0.003)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor = 0.001,patience = 5,verbose = True)

**weight initialization**

In [ ]:
#def weight_initialization():
    #for module in model.modules():
        #if isinstance(module,nn.Conv2d):
            #nn.init.kaiming_uniform_(module.weight)
            ##nn.init.constant_(module.bias,0.01)# xavier_normal_, xavier_uniform_, kaiming_normal_, kaiming_uniform_
            
        #if isinstance(module,nn.Linear):
            #nn.init.kaiming_uniform_(module.weight)
            ##nn.init.constant_(module.bias,0.01)
#weight_initialization()

In [ ]:
#??torch.optim.lr_scheduler.ReduceLROnPlateau

In [ ]:
steps = 40
loss_intitial = copy.deepcopy(model.cpu())

In [ ]:
train_plot = []
test_plot = []
acc_plot = []

In [ ]:
for epochs in range(epoch):
    model.train()
    train_loss =0
    test_loss = 0
    acc = 0
    test_total = 0
    for inputs,label in train_loader:
        optimizer.zero_grad()
        
        output = model(inputs)
        loss = criterion(output,label)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
    
    model.eval()
    for test_inputs,test_labels in test_loader:
        
        test_output = model(test_inputs)
        t_loss = criterion(test_output,test_labels)
        
        test_loss += t_loss.item() * test_inputs.size(0)
        
        _,pred = torch.max(test_output,1)
        t_acc = torch.sum(pred == test_labels.data)
        acc += t_acc
        
        test_total += len(test_labels)
        
        
    test_counter = test_loss / len(test_image)
    train_plot.append(train_loss/len(train_image))
    test_plot.append(test_counter)
    acc_plot.append(acc/len(test_image))
    scheduler.step(test_counter)
    
    print(f"Train {train_loss/len(train_image)}  Test: {test_counter} ACC: {acc/len(test_image)}")
    
    
    
        
    

In [ ]:
print(len(class_name))

In [ ]:
torch.save(model.state_dict(),"car_part_dedection.pt")


In [ ]:
model = torchvision.models.resnet50(pretrained = False)
model.fc = nn.Sequential(nn.Linear(2048,128),
                        nn.ReLU(inplace = True),
                        nn.Linear(128,len(class_name)))
model.load_state_dict(torch.load("car_part_dedection.pt"))


In [ ]:
model_final = copy.deepcopy(model.cpu())

In [ ]:
x,y = iter(train_loader).__next__()
metric = loss_landscapes.metrics.Loss(criterion,x,y)
loss_data = loss_landscapes.linear_interpolation(loss_intitial,model_final,metric,10,deepcopy_model = True)

loss_graph

In [ ]:
plt.plot([1/10 *i for i in range(10)],loss_data)
plt.title(" interpolasyon linear loss ")
plt.xlabel("interpolasyon katsayı")
plt.ylabel("loss")
axes = plt.gca()
plt.show()

In [ ]:
plt.plot(train_plot)
plt.plot(test_plot)
plt.plot(acc_plot)
plt.legend(["train loss","test loss","acc"])
plt.show()


In [ ]:
pred_folder = ["../input/automobilepartsindentification/Automobile-parts/Bevel-gear/image (101).jpg","../input/automobilepartsindentification/Automobile-parts/clutch/image (14).jpg","../input/automobilepartsindentification/Automobile-parts/piston/image (66).jpg","../input/automobilepartsindentification/Automobile-parts/shocker/image (22).jpeg","../input/automobilepartsindentification/Automobile-parts/cylincer/image (12).jpeg"]
names = ["bevel-gear","clutch","piston","shocker","cylincer"]

In [ ]:
pred_img_folder = [Image.open(a) for a in pred_folder]
pred_stacked = torch.stack([test_transform(b) for b in pred_img_folder])

In [ ]:
with torch.no_grad():
    model.eval()
    output = model(pred_stacked)
    output = F.softmax(output)
    

In [ ]:
fig,ax = plt.subplots(1,len(names),figsize = (20,5))
for i,img in enumerate(pred_img_folder):
    axs = ax[i]
    axs.axis("off")
    axs.set_title(f"Prediction: {class_name[torch.argmax(output[i])]}  Real: {names[i]}")
    axs.imshow(img)